In [1]:
import os
import sys
import time

In [2]:
cwd = !pwd
sys.path.append(os.path.split(cwd[0])[0])

In [3]:
from fantastic_amazon import amazon_scraper, sheets_connector
import threading

In [4]:
SERVICE_FILE = "/home/max/fantastic-amazon-c1ba8f85a23b.json"
SHEET_KEY = "1M8FT1qtKr5VgUAk-7IvQdX8Hp2Sn13uZLL3tq2o7YOU"
SHEET_NAME = "PRICES"
gs = sheets_connector.GoogleSheets(sheet_key=SHEET_KEY, sheet_name=SHEET_NAME, service_file=SERVICE_FILE)

In [5]:
PNs = gs.w.range("B3:B30")
url_dict = {} 
for PN in PNs:
    url_row = 'H' + str(PN[0].row) + ':J' + str(PN[0].row) 
    urls = gs.get_urls(url_row)
    url_dict[PN[0].value] = urls

In [6]:
def scraper_t(PN, url):
    if url:
        try:
            scrape_result = amazon_scraper.AmazonScrapper().scrape_product_page('amazon.com', url=url, headless=True)
            review_master_dict[PN][url] = scrape_result
        except Exception as e:
            print(e)

In [7]:
#chunking the list
n = 2
lst = list(url_dict.items())
review_master_dict = {PN: {url: {} for url in urls} for PN, urls in lst}
lst_flat = [(pn, _url) for pn, urls in lst for _url in urls ]
# iter_list = [lst[i:i + n] for i in range(0, len(lst), n)]
ts = []
for PN, url in lst_flat:
    while True:
        if not ts or len(ts) < n:
            break
        else:
            scripts_finished = [not t.is_alive() for t in ts]
            if any(scripts_finished):
                pop_i = [i for i, status in enumerate(scripts_finished) if status]
                ts[pop_i[0]].join()
                ts.pop(pop_i[0])
                break
            else:
                time.sleep(1) 
    t = threading.Thread(target=scraper_t, args=(PN, url))
    t.start()
    ts.append(t)
    print(PN, url) #, ts)

C31DUAL4KDP https://www.amazon.com/StarTech-com-2-Port-USB-C-DisplayPort-MST/dp/B01KIMP28Y/ref=sr_1_5?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-5
C31DUAL4KDP https://www.amazon.com/2-Port-DisplayPort-Adapter-Monitor-Splitter/dp/B07563JV7H/ref=sr_1_8?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-8
C31DUAL4KDP https://www.amazon.com/gofanco-DisplayPort-Displayport-Thunderbolt-Compatible/dp/B07MF4QXFX/ref=sr_1_3?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-3
C31DUAL4KHDMI https://www.amazon.com/StarTech-com-2-Port-USB-C-HDMI-MST/dp/B06XPVGQKY/ref=sr_1_6?dchild=1&keywords=USB-C+to+Dual+hdmi+Video+Adapter&qid=1621983776&sr=8-6
C31DUAL4KHDMI https://www.amazon.com/Anker-Portable-Supports-Compatible-Thunderbolt/dp/B0894B525X/ref=sr_1_9?dchild=1&keywords=USB-C+to+Dual+hdmi+Video+Adapter&qid=1621983776&sr=8-9
C31DUAL4KHDMI https://www.amazon.com/Sabrent-Adapter-Supports-Monitors-Compatible/dp/B0

In [8]:
import json

with open("product_info.json", 'w') as f:
    json.dump(review_master_dict, f)